In [ ]:
# import pickle

# training_data_path = "/scratch/data/open_teach/processed_data_pkl/grab_ball.pkl"

# with open(training_data_path, "rb") as f:
#     training_data = pickle.load(f)

# from pathlib import Path
# import imageio

# training_data.keys() # ['observations', 'max_cartesian', 'min_cartesian', 'max_gripper', 'min_gripper', 'task_emb']

# demo_num = 0

# max_cartesian = training_data['max_cartesian']
# min_cartesian = training_data['min_cartesian']
# max_gripper = training_data['max_gripper']
# min_gripper = training_data['min_gripper']

# print(f"max_cartesian: {max_cartesian}\nmin_cartesian: {min_cartesian}")
# len(training_data['observations']) # 100

# training_data['observations'][demo_num].keys() # ['demo_dir', 'pixels0', 'cartesian_states', 'gripper_states']

# cartesian_states = training_data['observations'][demo_num]['cartesian_states']
# gripper_states = training_data['observations'][demo_num]['gripper_states']

# frames = training_data['observations'][demo_num]['pixels0'][..., ::-1]
# video_path = Path(f"/home/danny/Downloads/demo_{demo_num}.mp4")
# video_path.parent.mkdir(parents=True, exist_ok=True)
# imageio.mimsave(video_path, frames, fps=30)

## Replay from Dataset sampled data

In [1]:
# preprocess the actions
import pickle
from pathlib import Path
import imageio
from p3po.read_data.p3po_xarm import BCDataset, get_relative_action, get_quaternion_orientation

path = "/scratch/data/open_teach/processed_data_pkl"
processed_path = "./processed_data"
tasks = ["grab_ball"]
num_demos_per_task = 100
obs_type = "features"
history = True
history_len = 100
prompt =  "text"
temporal_agg = False
num_future_actions = 1
img_size = 128 # should not matter
action_after_steps = 1
intermediate_goal_step = 30
store_actions = True
training_keys = ["graph"]
subsample = 1
skip_first_n = 0
relative_actions = True
debug_idx = 0

dataset = BCDataset(
    path=path,
    processed_path=processed_path,
    tasks=tasks,
    num_demos_per_task=num_demos_per_task,
    obs_type=obs_type,
    history=history,
    history_len=history_len,
    prompt=prompt,
    temporal_agg=temporal_agg,
    num_future_actions=num_future_actions,
    img_size=img_size,
    action_after_steps=action_after_steps,
    intermediate_goal_step=intermediate_goal_step,
    store_actions=store_actions,
    training_keys=training_keys,
    subsample=subsample,
    skip_first_n=skip_first_n,
    relative_actions=relative_actions,
    use_quaternion_orientation=True,
    debug_idx=debug_idx
)

Loading /scratch/data/open_teach/processed_data_pkl/grab_ball.pkl


In [2]:
from dm_env import specs
import pickle
import numpy as np
from p3po.replay_buffer import make_expert_replay_loader

action_spec = specs.BoundedArray(
    (dataset._max_action_dim,),
    np.float32,
    dataset.stats["actions"]["min"],
    dataset.stats["actions"]["max"],
    "action",
)

batch_size=1 # actual training used 64
expert_replay_loader = make_expert_replay_loader(dataset, batch_size=batch_size)
sample = next(iter(expert_replay_loader))
stats = expert_replay_loader.dataset.stats # stats passed to agent act function

In [ ]:
import pickle

pickle.dump(sample, open("debug/sample.pkl", "wb"))
pickle.dump(action_spec, open("debug/action_spec.pkl", "wb"))
pickle.dump(stats, open("debug/stats.pkl", "wb"))

: 

In [3]:
actions = sample["actions"] # B x history_len [x num_future_actions] x action_dim
proprioceptive = sample["proprioceptive"] # B x history_len x 8
graph = sample["graph"] # B x history_len x (n_points * 3)

# shape: B x num_future_actions x history_len x action_dim
actions = actions[0]

In [ ]:
from p3po.suite.ur5e_env import UR5Env

env = UR5Env(camera_names=["front_right"], include_depth=False, training=False, relative_actions=True)
env.reset()

TCP offset set to 0.1734126403712062. This is based on the z offset of the gripper. Please check if this is correct.
Gripper auto-calibrated to [3, 255]


In [ ]:
for i in range(len(actions)):
    action = actions[i]
    env.step(action)